In [1]:
%run init.ipynb

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
停用表配置成功


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left = 10,
    truncated_length_right = 40,
    filter_low_freq = 2
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/91 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.087 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 91/91 [00:03<00:00, 28.41it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 93/93 [00:00<00:00, 138.27it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 6

In [4]:
preprocessor.context

{'embedding_input_dim': 177,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x14700400a90>,
 'vocab_size': 177,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x14709fc6ba8>}

In [5]:
# glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
# term_index = preprocessor.context['vocab_unit'].state['term_index']
# embedding_matrix = glove_embedding.build_matrix(term_index)
# l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
# embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed
)

In [7]:
padding_callback = mz.models.KNRM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

In [8]:
model = mz.models.KNRM()

model.params['task'] = ranking_task
# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

KNRM(
  (embedding): Embedding(177, 100, padding_idx=0)
  (kernels): ModuleList(
    (0): GaussianKernel()
    (1): GaussianKernel()
    (2): GaussianKernel()
    (3): GaussianKernel()
    (4): GaussianKernel()
    (5): GaussianKernel()
    (6): GaussianKernel()
    (7): GaussianKernel()
    (8): GaussianKernel()
    (9): GaussianKernel()
    (10): GaussianKernel()
    (11): GaussianKernel()
    (12): GaussianKernel()
    (13): GaussianKernel()
    (14): GaussianKernel()
    (15): GaussianKernel()
    (16): GaussianKernel()
    (17): GaussianKernel()
    (18): GaussianKernel()
    (19): GaussianKernel()
    (20): GaussianKernel()
  )
  (out): Linear(in_features=21, out_features=1, bias=True)
)
Trainable params:  17722


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-1 Loss-0.960]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-2 Loss-0.950]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-3 Loss-0.937]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-4 Loss-0.922]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-5 Loss-0.905]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-6 Loss-0.883]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-7 Loss-0.858]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-8 Loss-0.827]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-9 Loss-0.790]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941



[Iter-10 Loss-0.746]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2941 - normalized_discounted_cumulative_gain@5(0.0): 0.2941 - mean_average_precision(0.0): 0.2941

Cost time: 3.7208666801452637s


In [12]:
import gc
gc.collect()

0